In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, DenseNet121, EfficientNetB0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.layers import Input
from sklearn.ensemble import VotingClassifier

# Define ImageDataGenerators for training and validation with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    brightness_range=[0.2, 1.5]
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    r"C:\Users\Priyamvadha Pradeep\Desktop\FYP\CNN\New Models\train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  # Two classes: ALS and Healthy
)

val_generator = val_datagen.flow_from_directory(
    r"C:\Users\Priyamvadha Pradeep\Desktop\FYP\CNN\New Models\validate",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Define the models (ResNet50, DenseNet121, EfficientNetB0)
def create_resnet50_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_densenet121_model():
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_efficientnet_model():
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Initialize models
resnet50_model = create_resnet50_model()
densenet121_model = create_densenet121_model()
efficientnet_model = create_efficientnet_model()

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)

# Train models
history_resnet50 = resnet50_model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)

history_densenet121 = densenet121_model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)

history_efficientnet = efficientnet_model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)

# Get predictions from all models
y_pred_resnet50 = resnet50_model.predict(val_generator)
y_pred_densenet121 = densenet121_model.predict(val_generator)
y_pred_efficientnet = efficientnet_model.predict(val_generator)

# Combine the predictions using soft voting (average the probabilities)
y_pred_avg = (y_pred_resnet50 + y_pred_densenet121 + y_pred_efficientnet) / 3
y_pred_avg = (y_pred_avg > 0.5).astype(int)  # Convert probabilities to class labels

# Get true labels
y_true = val_generator.classes

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred_avg)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report
report = classification_report(y_true, y_pred_avg, target_names=train_generator.class_indices.keys())
print("Classification Report:")
print(report)

Found 978 images belonging to 2 classes.
Found 246 images belonging to 2 classes.
Epoch 1/30


c:\Users\Priyamvadha Pradeep\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


31/31 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - accuracy: 0.5864 - loss: 0.7406 - val_accuracy: 0.6667 - val_loss: 0.6455 - learning_rate: 1.0000e-04
Epoch 2/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 64s 2s/step - accuracy: 0.6493 - loss: 0.6504 - val_accuracy: 0.6667 - val_loss: 0.6393 - learning_rate: 1.0000e-04
Epoch 3/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.6515 - loss: 0.6761 - val_accuracy: 0.6667 - val_loss: 0.6396 - learning_rate: 1.0000e-04
Epoch 4/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 70s 2s/step - accuracy: 0.6567 - loss: 0.6485 - val_accuracy: 0.6667 - val_loss: 0.6381 - learning_rate: 1.0000e-04
Epoch 5/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 69s 2s/step - accuracy: 0.6867 - loss: 0.6284 - val_accuracy: 0.6667 - val_loss: 0.6401 - learning_rate: 1.0000e-04
Epoch 6/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.6511 - loss: 0.6582 - val_accuracy: 0.6667 - val_loss: 0.6423 - learning_rate: 1.0000e-04
Epoch 7/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 64s 2s/step - accuracy: 0.6776 - loss: 0.6363 - v

c:\Users\Priyamvadha Pradeep\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Priyamvadha Pradeep\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Priyamvadha Pradeep\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

In [4]:
from sklearn.metrics import accuracy_score

# Calculate and print accuracies for each individual model
accuracy_resnet50 = accuracy_score(y_true, (y_pred_resnet50 > 0.5).astype(int))
accuracy_densenet121 = accuracy_score(y_true, (y_pred_densenet121 > 0.5).astype(int))
accuracy_efficientnet = accuracy_score(y_true, (y_pred_efficientnet > 0.5).astype(int))

# Calculate and print accuracy for the ensemble model
accuracy_avg = accuracy_score(y_true, y_pred_avg)

# Print accuracies
print(f"ResNet50 Accuracy: {accuracy_resnet50:.4f}")
print(f"DenseNet121 Accuracy: {accuracy_densenet121:.4f}")
print(f"EfficientNetB0 Accuracy: {accuracy_efficientnet:.4f}")
print(f"Ensemble Model Accuracy (Soft Voting): {accuracy_avg:.4f}")

ResNet50 Accuracy: 0.6667
DenseNet121 Accuracy: 0.6789
EfficientNetB0 Accuracy: 0.6667
Ensemble Model Accuracy (Soft Voting): 0.6667
